Code to fetch latest historical weather data, starting 01-01-2025, up to 'now'

open-meteo.com provides hourly weather data, with high resolution. with hourly updates. predictions up to 16 days into to the future, as well as historical data.. 

You can access past weather data dating back to 1940 with the historical weather API 
offered. *However, there is a 5-day delay in the data*. If you want information for the most recent days, you can use the forecast API and adjust the Past Days setting.


https://open-meteo.com/en/docs/knmi-api?models=knmi_seamless (forecast API) for OBSERVED weather data up to 3 months  ago, up to 'current' (actual) weather
https://open-meteo.com/en/docs/historical-weather-api for weather data > 3 months  ago (secondary, only needs to be run when weather data was not fetched >3 months)


In [1]:
# install related packages

%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# weather features and model involved
temperature_2m 🌡️
wind_speed_10m 💨
wind_speed_100m 💨
cloud_cover ☁️
snowfall ☃
apparent_temperature 🥵
radiation features (3x): diffuse_radiation,direct_normal_irradiance,shortwave_radiation
(Diffuse Solar Radiation DHI, Direct Normal Irradiance DNI, Shortwave Solar Radiation GHI)

Location DeBilt ; model used "KNMI-seamless"

In [ ]:
'''# historical API to fetch historical data (ran once, and then commented out)

import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
from datetime import datetime, timezone, timedelta

# define now
now = datetime.now(timezone.utc)
# calculate 5 days back from now
end_date = now - timedelta(days=5)
end_date_date_str = end_date.strftime("%Y-%m-%d")
print(f"5 days back from today: {end_date_date_str}")


# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.12949,
	"longitude": 5.20514,
	"hourly": ["temperature_2m", "wind_speed_10m", "apparent_temperature", "cloud_cover", "snowfall", 
            "diffuse_radiation", "direct_normal_irradiance", "shortwave_radiation"],
	"models": "knmi_seamless",
	"end_date": end_date_date_str,
	"start_date": "2025-01-01",    
}
responses = openmeteo.weather_api(url, params=params)

# Process single location. (would have to add a for-loop for multiple locations or weather models)
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(2).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(3).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(4).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(5).ValuesAsNumpy()
hourly_direct_normal_irradiance = hourly.Variables(6).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m_obs"] = hourly_temperature_2m
hourly_data["wind_speed_10m_obs"] = hourly_wind_speed_10m
hourly_data["apparent_temperature_obs"] = hourly_apparent_temperature
hourly_data["cloud_cover_obs"] = hourly_cloud_cover
hourly_data["snowfall_obs"] = hourly_snowfall
hourly_data["diffuse_radiation_obs"] = hourly_diffuse_radiation
hourly_data["direct_normal_irradiance_obs"] = hourly_direct_normal_irradiance
hourly_data["shortwave_radiation_obs"] = hourly_shortwave_radiation

weather_dataframe_obs = pd.DataFrame(data = hourly_data)
print(weather_dataframe_obs)

import sqlite3

# Connect to the SQLite database
db_path = '../data/WARP.db'
conn = sqlite3.connect(db_path)

# Write the DataFrame to the database table 'raw_weather_obs'
# If table exists, replace it. If not, create new table
weather_dataframe_obs.to_sql('raw_weather_obs', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print("Data successfully written to database table 'raw_weather_obs'")

In [6]:
import sqlite3  # moved over to py file
import pandas as pd

# Connect to the SQLite database
db_path = '../data/WARP.db'
conn = sqlite3.connect(db_path)

# Connect to the SQLite database using the existing db_path
conn = sqlite3.connect(db_path)
# Query the last 5 rows from raw_weather_obs table
df_head = pd.read_sql_query("SELECT * FROM raw_weather_obs ORDER BY date DESC LIMIT 5", conn)

# Close the connection
conn.close()

# Display the results
print(df_head)

# Connect to the SQLite database
# collect the date column from the raw_weather_obs table
conn = sqlite3.connect(db_path)
hist_weather_dates = pd.read_sql_query("SELECT date FROM raw_weather_obs", conn)
conn.close()

# Convert to datetime and find max date
hist_weather_dates['date'] = pd.to_datetime(hist_weather_dates['date'])
most_recent_date = hist_weather_dates['date'].max().strftime('%Y-%m-%d')
print(f"The most recent date in raw_weather_obs is: {most_recent_date}")

                        date  temperature_2m_obs  wind_speed_10m_obs  \
0  2025-05-09 10:00:00+00:00           17.932499                8.28   
1  2025-05-09 09:00:00+00:00           16.782499                7.20   
2  2025-05-09 08:00:00+00:00           15.082500                6.48   
3  2025-05-09 07:00:00+00:00           12.282500                5.76   
4  2025-05-09 06:00:00+00:00            9.082500                5.40   

   apparent_temperature_obs  cloud_cover_obs  snowfall_obs  \
0                 15.761242             98.0           0.0   
1                 14.514784             84.0           0.0   
2                 12.626129              0.0           0.0   
3                 10.021635              0.0           0.0   
4                  6.732677              0.0           0.0   

   diffuse_radiation_obs  direct_normal_irradiance_obs  \
0             190.664032                    717.224670   
1             166.761307                    713.572327   
2             149.19

In [2]:
''' does not work:# forecast API to fetch forecast data (py file based)
! python3 ingest_open-meteo_obs.py

SyntaxError: incomplete input (2779817388.py, line 1)

In [7]:
# forecast API to fetch 'recent' historical data

import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
from datetime import datetime, timezone, timedelta

# define now
now = datetime.now(timezone.utc)
# Convert current time to YYYY-MM-DD format
now_date_str = now.strftime("%Y-%m-%d")

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.12949,
	"longitude": 5.20514,
	"hourly": ["temperature_2m", "wind_speed_10m", "apparent_temperature", "cloud_cover", "snowfall", "diffuse_radiation", "direct_normal_irradiance", "shortwave_radiation"],
	"models": "knmi_seamless",
	"end_date": now_date_str,
	"start_date": most_recent_date,
}
responses = openmeteo.weather_api(url, params=params)

# Process single location. (would have to add a for-loop for multiple locations or weather models)
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(2).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(3).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(4).ValuesAsNumpy()
hourly_diffuse_radiation = hourly.Variables(5).ValuesAsNumpy()
hourly_direct_normal_irradiance = hourly.Variables(6).ValuesAsNumpy()
hourly_shortwave_radiation = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m_obs"] = hourly_temperature_2m
hourly_data["wind_speed_10m_obs"] = hourly_wind_speed_10m
hourly_data["apparent_temperature_obs"] = hourly_apparent_temperature
hourly_data["cloud_cover_obs"] = hourly_cloud_cover
hourly_data["snowfall_obs"] = hourly_snowfall
hourly_data["diffuse_radiation_obs"] = hourly_diffuse_radiation
hourly_data["direct_normal_irradiance_obs"] = hourly_direct_normal_irradiance
hourly_data["shortwave_radiation_obs"] = hourly_shortwave_radiation

recent_obs_dataframe = pd.DataFrame(data = hourly_data)
# Filter out future dates (greater than 'now'), so dataset is not contaminated with predictive data
recent_obs_dataframe = recent_obs_dataframe[recent_obs_dataframe['date'] <= now]
print(recent_obs_dataframe)


Coordinates 52.13199996948242°N 5.190999984741211°E
Elevation 5.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s
                        date  temperature_2m_obs  wind_speed_10m_obs  \
0  2025-05-09 00:00:00+00:00            9.082500           11.159999   
1  2025-05-09 01:00:00+00:00            8.282500            9.000000   
2  2025-05-09 02:00:00+00:00            7.432500            7.559999   
3  2025-05-09 03:00:00+00:00            7.182500            6.840000   
4  2025-05-09 04:00:00+00:00            6.132500            6.480000   
5  2025-05-09 05:00:00+00:00            6.832500            6.840000   
6  2025-05-09 06:00:00+00:00            9.082500            5.400000   
7  2025-05-09 07:00:00+00:00           12.282500            5.760000   
8  2025-05-09 08:00:00+00:00           14.782500            5.040000   
9  2025-05-09 09:00:00+00:00           16.832499            5.760000   
10 2025-05-09 10:00:00+00:00           17.882500            8.640000   
11 2025-05-09

In [8]:
import sqlite3 
import pandas as pd

# Connect to the SQLite database
db_path = '../data/WARP.db'
conn = sqlite3.connect(db_path)

# Read existing data from raw_weather_obs
existing_data = pd.read_sql_query("SELECT * FROM raw_weather_obs", conn)

# Convert date columns to datetime for both dataframes
existing_data['date'] = pd.to_datetime(existing_data['date'])
recent_obs_dataframe['date'] = pd.to_datetime(recent_obs_dataframe['date'])

# Remove any duplicates based on date and keep the latest values
merged_df_weather_obs = pd.concat([existing_data, recent_obs_dataframe])
initial_rows = len(merged_df_weather_obs)
merged_df_weather_obs = merged_df_weather_obs.drop_duplicates(subset='date', keep='last')
removed_rows = initial_rows - len(merged_df_weather_obs)
print(f"Removed {removed_rows} duplicate rows")

# Sort by date
merged_df_weather_obs = merged_df_weather_obs.sort_values('date')

# Write the merged dataframe back to the database
merged_df_weather_obs.to_sql('raw_weather_obs', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print(f"Data successfully merged. Total rows: {len(merged_df_weather_obs)}")
print(f"Date range: {merged_df_weather_obs['date'].min()} to {merged_df_weather_obs['date'].max()}")

Removed 11 duplicate rows
Data successfully merged. Total rows: 3084
Date range: 2025-01-01 00:00:00+00:00 to 2025-05-09 11:00:00+00:00
